Сначала импортируем необходимые модули

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
from concurrent.futures import ThreadPoolExecutor

Задаем переменные для ссылок.  
url - основная ссылка   
urlstub - ссылка для прикрепления аттрибутов href.

In [ ]:
urlstub = 'https://www.wdl.org'
url = 'https://www.wdl.org/ru/search/?item_type=print-photograph#1'

Создаем запрос на нашу главную ссылку и делаем из ответа BeautifulSoup.  
Извлекаем из него необходимые для нас ссылки, присваиваем списку ссылок имя titles.

In [ ]:
r = requests.get(url)
bs = BeautifulSoup(r.text, 'html.parser')
titles = bs.select('a.title')
next_page = 1

Создаем две фунции.  
title_parser берет в качестве аргумента текущую страницу:  
   1.находит на ней все ссылки на фотографии и группирует их в лист link_list.
   2.создает csv файл pictures_* - соответствует номеру страницы
   3.проходится по каждой ссылке в link_list и записывает нужную нам информацию в созданный csv файл, применяя к каждой ссылке функцию picture_compile.
Здесь мы используем ThreadPoolExecutor, чтобы обрабатывать несколько запросов сразу, что многократно ускоряет сбор информации.

In [ ]:
 def picture_compile(link, csv_file):
    r = requests.get(link)
    bs = BeautifulSoup(r.text, 'html.parser')
    r.close()
    picture = {}
    picture['Название'] = bs.select_one('h1#page-title').text.strip()
    picture['Время создания'] = bs.select_one('time[itemprop="dateCreated"]').attrs['datetime']
    picture['Период'] = bs.select_one('div#subject-date time').attrs['datetime']
    places = bs.select('div#places ul li')
    picture['Место'] = [place.text.strip().replace('\n', ', ') for place in places]
    csv_columns = ['Название', 'Время создания', 'Период', 'Место']
    with open(csv_file, 'a', encoding='utf-8') as pic:
        writer = csv.DictWriter(pic, fieldnames=csv_columns)
        writer.writerow(picture)
        
def title_parser(title_list):
    csv_columns = ['Название', 'Время создания', 'Период', 'Место']
    csv_file = 'pictures_' + str(next_page) +'.csv'
    with open(csv_file, 'a', encoding='utf-8') as pic:
        writer = csv.DictWriter(pic, fieldnames=csv_columns)
        writer.writeheader()
   
    link_list = []
    for title in title_list:
        photolink = urlstub + title.attrs['href']
        link_list.append(photolink)
    processes = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        for link in link_list:
            processes.append(executor.submit(picture_compile, link, csv_file))
      
    print('Сохранена страница: ', next_page)
    time.sleep(3.0)

Запускаем эту функцию с нашим листом ссылок.

In [ ]:
title_parser(titles)

Выводим для пользователя количество сохраненных ссылок.  
Генерируем ссылку для следующей страницы.

In [ ]:
total_saved = len(titles)
print('сохранено ' + str(total_saved) + ' фото')
next_page += 1
next_stub = 'https://www.wdl.org/ru/search/?item_type=print-photograph&page='
next_link = next_stub +str(next_page)

Дальше продолжаем загружать страницы и переходить по ссылкам с фотографиями, извлекая необходимую информацию.
Страницы будут загружаться до тех пор, пока не останется новых страниц.
Если хотим загрузить меньше страниц, то раскомменчиваем фрагмент и указываем нужное число страниц (на одну страницу приходится 42 фото):
if next_page == 6:  
&nbsp;&nbsp;&nbsp;&nbsp;   print('Сохранено пять страниц, хватит')  
&nbsp;&nbsp;&nbsp;&nbsp;   break  
Если этих строчек нет, то будут сохранены данные обо всех фотографиях на примерно 200 страницах.

In [ ]:
while True:
    r = requests.get(next_link)
    if r.status_code != 200:
        print('Следующей страницы не существует')
        break
    bs = BeautifulSoup(r.text, 'html.parser')
    titles = bs.select('a.title')
    title_parser(titles)
    total_saved += len(titles)
    print('сохранено ' + str(total_saved) + ' фото')

    next_page += 1
#     if next_page == 6:
#         print('Сохранено пять страниц, хватит')
#         break
    next_link = next_stub + str(next_page)

Соединяем все собранные csv файлы в один (называем его здесь pictures.csv).
В первом файле записываем всё, включая заголовок, в остальных файлах заголовок пропускаем.

In [ ]:
total = open('pictures.csv', 'a', encoding='utf8', newline='')
writer = csv.writer(total)

with open('pictures_1.csv', encoding='utf8') as f:
    reader = csv.reader(f)
    for line in reader:
        if line:
            writer.writerow(line)

for i in range(2, next_page):
    with open('pictures_'+str(i)+'.csv', encoding='utf8') as f:
        reader = csv.reader(f)
        next(reader) 
        for line in reader:
            if line and line!='':
                writer.writerow(line)
    print(i, sep=', ')

total.close()